# Huffman树的构建

In [3]:
#定义节点的数据结构
class HuffmanNode:
    def __init__(self,word_id,frequency):
        self.word_id = word_id # 每个词一个id
        self.frequency = frequency # 出现的频率
        self.left_child = None
        self.right_child = None
        self.father = None #他爹
        self.Huffman_code = [] #往左是0往右是1
        self.path = [] # 从根节点到这个节点经过的Id

In [4]:
wordid_frequency_dict = {0: 4, 1: 6, 2: 3, 3: 2, 4: 2} # 0词频率4...
wordid_code = dict()
wordid_path = dict()
unmerge_node_list = [HuffmanNode(wordid,frequency) for wordid,frequency in wordid_frequency_dict.items()] 
#还没合并的节点
huffman = [HuffmanNode(wordid,frequency) for wordid,frequency in wordid_frequency_dict.items()]
#先把所有节点放进去

In [5]:
def merge_node(node1,node2):
    sum_frequency = node1.frequency + node2.frequency
    mid_node_id = len(huffman)
    father_node = HuffmanNode(mid_node_id, sum_frequency) #初始化父亲节点，这个树是从下到上去构建的
    if node1.frequency >= node2.frequency: #频率大的左边
        father_node.left_child = node1
        father_node.right_child = node2
    else:
        father_node.left_child = node2
        father_node.right_child = node1
    huffman.append(father_node)
    return father_node

In [6]:
#node = merge_node(unmerge_node_list[0],unmerge_node_list[1])

In [7]:
#node.frequency

In [8]:
# node_list是把未合并的那个unmerge_node_list给传进来
def build_tree(node_list):
    while len(node_list) > 1: #未合并节点大于1就可以继续合并
        i1 = 0  # 概率最小的节点
        i2 = 1  # 概率第二小的节点
        #这里是排序
        if node_list[i2].frequency < node_list[i1].frequency:
            [i1, i2] = [i2, i1]
        for i in range(2, len(node_list)):
            if node_list[i].frequency < node_list[i2].frequency:
                i2 = i
                if node_list[i2].frequency < node_list[i1].frequency:
                    [i1, i2] = [i2, i1]
        #然后合并最小的两个节点，然后判断左右
        father_node = merge_node(node_list[i1], node_list[i2])  # 合并最小的两个节点
        #先删大的，再删小的
        if i1 < i2:
            node_list.pop(i2)
            node_list.pop(i1)
        elif i1 > i2:
            node_list.pop(i1)
            node_list.pop(i2)
        else:
            raise RuntimeError('i1 should not be equal to i2')
        #删除了之后再插入父亲节点，最后剩下唯一的根节点
        node_list.insert(0, father_node)  # 插入新节点
    root = node_list[0]
    return root

In [9]:
root = build_tree(unmerge_node_list)

In [10]:
len(huffman) 

9

In [11]:
def generate_huffman_code_and_path():
    stack = [root]
    while len(stack) > 0:
        node = stack.pop()
        # 顺着左子树走
        # 有左必有右
        # 这个循环停止的时候必然是一个叶子节点。
        # 所有的叶子节点都是真正的单词节点
        while node.left_child or node.right_child:
            code = node.Huffman_code
            path = node.path
            node.left_child.Huffman_code = code + [1] # 往左是1
            node.right_child.Huffman_code = code + [0] # 往右是0
            node.left_child.path = path + [node.word_id]
            node.right_child.path = path + [node.word_id]
            # 把没走过的右子树加入栈
            stack.append(node.right_child)
            node = node.left_child
        # 把这些单词节点的word_id，Huffman_code，path读取出来，然后再把这些写进去huffman中单词节点的结构中
        word_id = node.word_id
        word_code = node.Huffman_code
        word_path = node.path
        huffman[word_id].Huffman_code = word_code
        huffman[word_id].path = word_path
        # 把节点计算得到的霍夫曼码、路径  写入词典的数值中
        # worid_code中的word_id位置和huffman[word_id]是一样的
        wordid_code[word_id] = word_code # 这里是0101
        wordid_path[word_id] = word_path # 这里是经过哪些节点
        # 最后返回这些真正的单词节点
    return wordid_code, wordid_path

In [12]:
[]+[1]+[0]

[1, 0]

In [13]:
wordid_code, wordid_path = generate_huffman_code_and_path()

In [14]:
wordid_code

{0: [1, 0], 1: [1, 1], 2: [0, 0], 3: [0, 1, 1], 4: [0, 1, 0]}

In [ ]:
def get_all_pos_and_neg_path():
    positive = []  # 所有词的正向路径数组
    negative = []  # 所有词的负向路径数组 
    for word_id in range(len(wordid_frequency_dict)):
        pos_id = []  # 存放一个词 路径中的正向节点id
        neg_id = []  # 存放一个词 路径中的负向节点id
        for i, code in enumerate(huffman[word_id].Huffman_code): #对于每个word_id对应的哈夫曼编码进行遍历
            if code == 1:
                pos_id.append(huffman[word_id].path[i]) # 中心词乘以这个节点的向量是pos
            else:
                neg_id.append(huffman[word_id].path[i])
        positive.append(pos_id)
        negative.append(neg_id)
    return positive, negative

In [18]:
huffman[0].path

[8, 7]

In [19]:
huffman[0].Huffman_code

[1, 0]

In [ ]:
positive, negative = get_all_pos_and_neg_path()

In [ ]:
#对于word_id为1的词，走8和7都是pos
positive

[[8], [8, 7], [], [6, 5], [6]]

In [ ]:
negative

[[7], [], [8, 6], [8], [8, 5]]

每个词作为周围词，他的正样本和负样本是怎么样的。比如上门那个例子，0这个点作为周围词，的正样本的周围词是8，负样本周围词是7

比如中心词是0，周围词是4